# Notebook for building and testing the model     

In this notebook, I will build and run an initial test over the first 2,048 rows of the data to ensure proper functionality before deploying on midway2 to train on GPUs. 

In [179]:
import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import save_image

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import sys

In [9]:
# cuda setup
device = torch.device("cuda")
kwargs = {'num_workers': 2, 'pin_memory': True} 

In [90]:
# hyper params
batch_size = 64
latent_size = 10
epochs = 10

In [4]:
# reading in the data
dat = np.genfromtxt('traindata/trip-2021-07-28.csv', delimiter=',')[1:,]

In [54]:
# need to subset this in a conditional fashion (i.e., each gamma value, here class c, has 1000 data points)
gamma = np.unique(dat[:,1])
idx = [np.where(dat[:,1] == gamma[i]) for i in np.arange(len(gamma))]

In [79]:
# create an 80/20 split in each data set
temp = [train_test_split(dat[idx[i][0],0], dat[idx[i][0],2], test_size=0.2, random_state=42) for i in np.arange(len(gamma))]

# for each gamma value...
Xltrain = []
Xltest = []
altrain = []
altest = []
for t in np.arange(len(temp)):
    Xltrain.append(temp[t][0])
    Xltest.append(temp[t][1])
    altrain.append(temp[t][2])
    altest.append(temp[t][3])

In [91]:
torch.utils.data.DataLoader(TensorDataset(torch.from_numpy(Xltrain[0]), torch.from_numpy(altrain[0])), batch_size=batch_size, shuffle=True, **kwargs)

In [683]:
class BaselineNet(nn.Module):
    def __init__(self, hidden1):
        super().__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(2, hidden1),
            nn.ReLU(),
            nn.Linear(hidden1, hidden1),
            nn.ReLU(),
            nn.Linear(hidden1, 50), 
            nn.ReLU(),
            nn.Linear(50, 1),
        )
        # self.fc2 = nn.Sequential(
        #     nn.Linear(1, 2),
        #     nn.Tanh(),
        # )

        # self.predict = nn.Sequential(
        #     nn.Linear(10 + 2, 1), 
        # )
        self.double()

    def forward(self, x, g):
        #x = self.fc1(x)
        #g = self.fc2(g)
        a = self.fc1(torch.cat((x,g), 1))
        return a

In [684]:
bnet = BaselineNet(500)
bnet.eval()
mse_loss = nn.MSELoss()
optimizer = optim.SGD(bnet.parameters(), lr=2e-4)

In [685]:
for e in range(800):
    #pred = bnet(torch.from_numpy(Xltrain[0]).float(), torch.from_numpy(np.repeat(gamma[0], len(Xltrain[0]))).float())
    #pred = bnet(torch.unsqueeze(torch.tensor(Xltrain[5]),1), torch.unsqueeze(torch.tensor(np.repeat(gamma[5], len(Xltrain[5]))),1))
    pred = bnet(torch.unsqueeze(torch.tensor(np.hstack((Xltrain[0:5]))),1), torch.unsqueeze(torch.tensor(np.repeat(gamma[0:5],len(Xltrain[0]))),1))
    step_loss = mse_loss(pred, torch.unsqueeze(torch.tensor(np.hstack((altrain[0:5]))),1))

    optimizer.zero_grad()
    step_loss.backward()
    # update with current step regression parameters 
    optimizer.step()

    if e % 100 == 0:
        print ('epoch [{}], Loss: {:.2f}'.format(e, step_loss.item()))

epoch [0], Loss: 2418022.70
epoch [100], Loss: nan
epoch [200], Loss: nan


KeyboardInterrupt: 

In [682]:
(bnet.fc1[4].weight)

Parameter containing:
tensor([[-0.0239, -0.0065,  0.0233,  ...,  0.0497,  0.0425,  0.0248],
        [ 0.0484, -0.0090,  0.0171,  ..., -0.0282, -0.0513,  0.0207],
        [-0.0232,  0.0248, -0.0247,  ..., -0.0252, -0.0024, -0.0178],
        ...,
        [ 0.0487,  0.0155,  0.0068,  ..., -0.0589, -0.0140, -0.0423],
        [-0.0306, -0.0460, -0.0129,  ..., -0.0064,  0.0208,  0.0207],
        [ 0.0654, -0.0045, -0.0395,  ..., -0.0121, -0.0151,  0.0022]],
       dtype=torch.float64, requires_grad=True)

In [649]:
r2_score(y_true=altrain[5], y_pred=pred.detach().numpy())
#bnet(torch.unsqueeze(torch.tensor(Xltrain[5][0:1]),1), torch.unsqueeze(torch.tensor(np.repeat(gamma[5],1)),1))
#torch.unsqueeze(torch.cat((torch.tensor(Xltrain[5]),torch.tensor(Xltrain[0])),0),1)
torch.unsqueeze(torch.tensor(np.repeat(gamma[0:9],len(Xltrain[0]))),1).shape

torch.Size([7200, 1])

In [481]:
#torch.unsqueeze(torch.tensor(Xltrain[0]),1)
#bnet.forward(torch.tensor(Xltrain[0]), torch.tensor(np.repeat(gamma[0], len(Xltrain[0]))))
torch.tensor(Xltrain[0]).view(-1, torch.tensor(Xltrain[0]).size(0)).shape
torch.unsqueeze(torch.tensor(Xltrain[0]),1)
#print(torch.unsqueeze(torch.linspace(-1,1,10),-1).shape)
#bnet.forward(torch.tensor(Xltrain[0][1]),torch.tensor(gamma[0]))
#torch.unsqueeze(torch.tensor(Xltrain[0]),0).shape

800

In [514]:
bnet.predict[0].weight

Parameter containing:
tensor([[ 71.3519, -73.1773, -42.9027,  71.6761,  72.8176, -74.0732, -73.8141]],
       dtype=torch.float64, requires_grad=True)